In [ ]:
import os
import pandas as pd
import geopandas as gp
from geoalchemy2 import Geometry, WKTElement
from shapely.geometry.multipoint import MultiPoint
from shapely.geometry.multilinestring import MultiLineString
from shapely.geometry.multipolygon import MultiPolygon

from gispy.oss import lst_ff
from gispy.sql.con import alchemy_engine
from gispy.sql.db import create_db
from gispy.geofiles import get_epsg_shp

In [ ]:
folder = '/home/useriam/gdata'

db = "workshop"

work_epsg = 3763

In [ ]:
# List ESRI Shapefiles in folder

files = lst_ff(folder, file_format='.shp')

In [ ]:
files

In [ ]:
# Create new database

create_db(db, overwrite=True, use_template=True)

In [ ]:
bgri = gp.read_file('/home/useriam/gdata/bgri.shp')

cols = bgri.columns.values

ccols = {}
for c in cols:
    if c.isupper():
        ccols[c] = c.lower()

bgri.rename(columns=ccols, inplace=True)

In [ ]:
bgri

In [ ]:
# Import data into PostgreSQL

# Get connector to database
conobj = alchemy_engine(db)

def sanitize_geom(geom, GeomClass, epsg):
    if geom.geom_type.startswith('Multi'):
        return WKTElement(geom.wkt, srid=epsg)
        
    else:
        return WKTElement(GeomClass([geom]).wkt, srid=epsg)

for f in files:
    # Check if we need to project data
    srs_code = get_epsg_shp(f)
    
    # Read data with GeoPandas
    gdf = gp.read_file(f)
    
    # Sanitize column names
    cols = gdf.columns.values

    ccols = {}
    for c in cols:
        if c.isupper():
            ccols[c] = c.lower()

    gdf.rename(columns=ccols, inplace=True)
    
    # Project if necessary
    if srs_code != work_epsg:
        print("We need to project {}. EPSG={}".format(f, str(srs_code)))
        gdf = gdf.to_crs('EPSG:{}'.format(str(work_epsg)))
    
    # Get Geometry Type
    gdf['geotype'] = gdf.geometry.geom_type
    
    geo_type = pd.DataFrame({
        'cgeom' : gdf.groupby(['geotype'])['geotype'].agg('count')
    }).reset_index().geotype.tolist()
    
    geo_type = geo_type[0] if len(geo_type) == 1 else [g for g in geo_type if g.startswith('Multi')][0]
    print(geo_type)
    
    # Add data to database
    if geo_type.startswith('Multi'):
        MultiClass = MultiPolygon if geo_type == 'MultiPolygon' else \
            MultiLineString if geo_type == 'MultiLineString' else MultiPoint
        
        gdf["geom"] = gdf.geometry.apply(lambda x: sanitize_geom(x, MultiClass, work_epsg))
    
    else:
        gdf["geom"] = gdf.geometry.apply(lambda x: WKTElement(x.wkt, srid=work_epsg))
    
    gdf.drop("geometry", axis=1, inplace=True)
    
    gdf.to_sql(
        os.path.splitext(os.path.basename(f))[0], conobj,
        if_exists='replace', index=False,
        dtype={"geom" : Geometry(geo_type, srid=work_epsg)}
    )